In [ ]:
!pip install shap

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import f_regression, f_classif
from imblearn.over_sampling import RandomOverSampler
import xgboost
from sklearn.ensemble import RandomForestClassifier
import shap
from sklearn.linear_model import Ridge
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.model_selection import KFold
from sklearn.feature_selection import RFECV

In [ ]:
data = pd.read_excel('1-s2.0-S2352340918305092-mmc2.xlsx', sheet_name = 'Dataset with Generated Features')
# data = pd.read_excel('augmented sample.xlsx')
data_composition = pd.read_excel('1-s2.0-S2352340918305092-mmc2.xlsx', sheet_name = 'DFT Calculated Dataset')

In [ ]:
#data = pd.read_excel('augmented sample.xlsx')

In [ ]:
data_join = data_composition.merge(data, 'inner', on='Material Composition')

In [ ]:
data_join.head(3)

,Material Composition,A site #1,A site #2,A site #3,B site #1,B site #2,B site #3,X site,Number of elements,energy_above_hull (meV/atom),formation_energy (eV/atom),index,goldschmidt_TF,goldschmidt_TF_ionic,octahedral_factor,octahedral_factor_ionic,A_O,B_O,A_B,num_of_atoms_host_Asite0,shannon_radii_host_Asite0,host_Asite0_Ionic Radius (angstroms),host_Asite0_Modulus of Elasticity 10³ Mpa,host_Asite0_ BP (K),host_Asite0_MP (K),host_Asite0_Density,host_Asite0_at. wt.,host_Asite0_BCCefflatcnt,host_Asite0_BCCenergy_pa,host_Asite0_BCCenergydiff,host_Asite0_BCCfermi,host_Asite0_BCCmagmom,host_Asite0_BCCvolume_pa,host_Asite0_BCCvolume_padiff,host_Asite0_GSbandgap,host_Asite0_GSefflatcnt,host_Asite0_GSenergy_pa,host_Asite0_GSmagmom,host_Asite0_GSvolume_pa,host_Asite0_HHIp,...,Bsite_IsBCC_range,Bsite_IsCubic_range,Bsite_IsFCC_range,Bsite_IsOrthorhombic_range,Bsite_IsRhombohedral_range,Bsite_IsMonoclinic_range,Bsite_IsTetragonal_range,Bsite_IsAlkali_range,Bsite_IsAlkalineEarth_range,Bsite_IsBoron_range,Bsite_IsCarbon_range,Bsite_IsChalcogen_range,Bsite_IsHalogen_range,Bsite_IsHydrogen_range,Bsite_IsNoblegas_range,Bsite_IsPnictide_range,Bsite_IsRareEarth_range,Bsite_IsTransition Metal_range,Bsite_OrbitalS_range,Bsite_OrbitalP_range,Bsite_OrbitalD_range,Bsite_OrbitalF_range,Bsite_Most stable oxidation State_range,Bsite_IsAlkali_range.1,Bsite_IsDBlock_range,Bsite_IsFBlock_range,Bsite_IsMetal_range,Bsite_IsMetalloid_range,Bsite_IsNonmetal_range,Bsite_NdUnfilled_range,Bsite_NdValence_range,Bsite_NfUnfilled_range,Bsite_NfValence_range,Bsite_NpUnfilled_range,Bsite_NpValence_range,Bsite_NsUnfilled_range,Bsite_NsValence_range,Bsite_NUnfilled_range,EnergyAboveHull,Formation_energy
0,Ba1Sr7V8O24,Ba,Sr,NaN,V,NaN,NaN,O,4,29.747707,-2.113335,1,1.021823,0.976828,0.414286,0.385714,2.86125,1.980,2.04125,7,1.44,1.26,15.0,1655,1042.0,2.54,87.6200,9.459886,-1.677940,0.004874,1.710570,0.0,52.91,-1.320,0,9.537909,-1.682814,0,54.230,420,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,29.747707,-2.113335
1,Ba2Bi2Pr4Co8O24,Ba,Bi,Pr,Co,NaN,NaN,O,5,106.702335,-1.311863,2,0.987385,0.889057,0.378571,0.464286,2.69500,1.930,1.82500,4,1.20,1.13,50.0,3785,1204.0,6.77,140.9077,8.311852,-4.648540,0.125290,3.592024,0.0,35.89,0.215,0,8.295222,-4.773830,0,35.675,950,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,106.702335,-1.311863
2,Ba2Ca6Fe8O24,Ba,Ca,NaN,Fe,NaN,NaN,O,4,171.608093,-1.435607,3,0.976009,0.908360,0.452857,0.392857,2.80750,2.034,2.04150,6,1.34,1.00,21.0,1757,1112.0,1.55,40.0780,8.383035,-1.927259,0.021070,2.895424,0.0,36.82,-0.950,0,8.454521,-1.948330,0,37.770,390,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,171.608093,-1.435607


In [ ]:
len(data)

1929

In [ ]:
features_full = data.drop(['index', 'Material Composition', 'EnergyAboveHull', 'Formation_energy'],axis=1)
target_value = data[['EnergyAboveHull']]
target_class = data[['EnergyAboveHull']]
target_class[target_class <= 40] = 1
target_class[target_class > 40] = 0


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


### 1. Feature Selection

#### (1) Drop constant features

In [ ]:
number_of_unique = features_full.nunique(axis = 0)
stat = pd.DataFrame(number_of_unique, columns=['number_of_uniques'])

const_var = stat.iloc[np.where(stat['number_of_uniques']==1)[0], :].index
feature_wo_const = features_full.drop(const_var, axis = 1)

In [ ]:
X_train, X_test, class_train, class_test = train_test_split(feature_wo_const, target_class, test_size=0.3, random_state=42)
X_train, X_test, value_train, value_test = train_test_split(feature_wo_const, target_value, test_size=0.3, random_state=42)

#### (2) Univariate Statistic

In [ ]:
def uncorr_features(X_train, y_train, threshold_of_p_value, for_regression=True):
    # Params:
    #        X_train: dataframe
    #        y_train: dataframe
    #        threshold_of_p_value: float, range(0.0, 1.0)
    
    # Return: 
    #        list of features that should be removed
    if for_regression:
        f_value, p_value = f_regression(X_train, y_train, center=True)
    else:
        f_value, p_value = f_classif(X_train, y_train)
    uni_FR = pd.DataFrame(np.concatenate([f_value.reshape(-1,1), p_value.reshape(-1,1)], axis=1), 
                      columns = ['f_value', 'p_value'], 
                      index = X_train.columns.tolist())
    #boxplot = uni_FR.boxplot(column=['p_value'],return_type=None)
    features_removed = uni_FR[uni_FR['p_value']>threshold_of_p_value].index
    return features_removed.tolist()
features_uni_value = uncorr_features(X_train, value_train, 0.1, for_regression=True)
features_uni_class = uncorr_features(X_train, class_train, 0.05, for_regression=False)

X_train_uni_value = X_train.drop(features_uni_value, axis = 1)
X_train_uni_class = X_train.drop(features_uni_class, axis = 1)

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


In [ ]:
print(X_train_uni_value.shape, X_train_uni_class.shape)

(1350, 681) (1350, 621)


#### (3) SHAP

In [ ]:
#shap.initjs()

# For classification
random_forest = RandomForestClassifier(random_state = 42)
random_forest.fit(X_train_uni_class, class_train['EnergyAboveHull'].ravel())

explainer = shap.TreeExplainer(random_forest)
shap_values = explainer.shap_values(X_train_uni_class) #num of class * num of rows* num of features

shap_value_mean = np.sum(np.abs(shap_values[0]), axis = 0)/len(shap_values[0]) + np.sum(np.abs(shap_values[1]), axis = 0)/len(shap_values[1])
shap_value_df = pd.DataFrame(shap_value_mean, index= X_train_uni_class.columns.tolist(), columns=['mean_shap_value'])
shap_value_df.sort_values(by=['mean_shap_value'], ascending=False)

features_shap_class = shap_value_df[shap_value_df['mean_shap_value']<=shap_value_df.quantile(0.75)[0]].index.tolist()

# For regression

Linear_Ridge = Ridge(random_state=42)
Linear_Ridge.fit(X_train_uni_value, value_train)

explainer_Ridge = shap.LinearExplainer(Linear_Ridge, X_train_uni_value, feature_perturbation="interventional")
shap_values_Ridge = explainer_Ridge.shap_values(X_train_uni_value) #num of class * num of rows* num of features

shap_value_mean_linear = np.sum(np.abs(shap_values_Ridge), axis = 0)/len(shap_values_Ridge)
shap_value_df_linear = pd.DataFrame(shap_value_mean_linear, index= X_train_uni_value.columns.tolist(), columns=['mean_shap_value'])
shap_value_df_linear.sort_values(by=['mean_shap_value'], ascending=False)

features_shap_value = shap_value_df_linear[shap_value_df_linear['mean_shap_value']<=shap_value_df_linear.quantile(0.75)[0]].index.tolist()

X_train_3_value = X_train_uni_value.drop(features_shap_value, axis = 1)
X_train_3_calss = X_train_uni_class.drop(features_shap_class, axis = 1)

The feature_perturbation option is now deprecated in favor of using the appropriate masker (maskers.Independent, or maskers.Impute)


In [ ]:
print(X_train_3_calss.shape, X_train_3_value.shape)

(1350, 155) (1350, 170)


#### (4) RFE

In [ ]:
decision_tree_classif = DecisionTreeClassifier(random_state=42)
rfecv_DTC = RFECV(estimator=decision_tree_classif, cv=KFold(n_splits = 5, random_state = 42, shuffle = True), scoring='f1', n_jobs = 4)
rfecv_DTC.fit(X_train_3_calss, class_train['EnergyAboveHull'].ravel())

features_selected = X_train_3_calss.columns[rfecv_DTC.support_].tolist()

X_train_4_class = X_train_3_calss[features_selected]

In [ ]:
decision_tree_reg = DecisionTreeRegressor(random_state=42)
rfecv_DTR = RFECV(estimator=decision_tree_reg, cv=KFold(n_splits = 5, random_state = 42, shuffle = True), scoring='r2', n_jobs = 4)

rfecv_DTR.fit(X_train_3_value, value_train['EnergyAboveHull'].ravel())
features_selected_value = X_train_3_value.columns[rfecv_DTR.support_].tolist()
X_train_4_value = X_train_3_value[features_selected_value]

In [ ]:
print(X_train_4_class.shape, X_train_4_value.shape)

(1350, 87) (1350, 24)


### 2. Baseline models

#### 2.1 SVM

##### (1) Regression

In [ ]:
from sklearn.svm import SVR
import numpy as np
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score, cross_validate, GridSearchCV, KFold, StratifiedKFold
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score, r2_score, mean_absolute_error, mean_squared_error

SVReg = make_pipeline(StandardScaler(), SVR())
param_grid_svr = {'svr__C': range(2230, 2270, 10)}

scoring_reg = {'R2': make_scorer(r2_score),  
           'MAE':make_scorer(mean_absolute_error),
           'MSE': make_scorer(mean_squared_error)}

grid_svr = GridSearchCV(SVReg, param_grid_svr, cv=KFold(5), return_train_score=True, scoring=scoring_reg, refit = 'R2')
grid_svr.fit(X_train_4_value, value_train.values.ravel())
grid_svr.score(X_test[features_selected_value], value_test)

0.6719511375183997

In [ ]:
grid_svr.best_params_

{'svr__C': 2260}

In [ ]:
value_test_predicted_svr = grid_svr.predict(X_test[features_selected_value])
print(r2_score(value_test, value_test_predicted_svr))
print(mean_absolute_error(value_test, value_test_predicted_svr))
print(np.sqrt(mean_squared_error(value_test, value_test_predicted_svr)))

0.7097145804324829
19.097498614254615
53.60122548505004


In [ ]:
#aug
value_test_predicted_svr = grid_svr.predict(X_test[features_selected_value])
print(r2_score(value_test, value_test_predicted_svr))
print(mean_absolute_error(value_test, value_test_predicted_svr))
print(np.sqrt(mean_squared_error(value_test, value_test_predicted_svr)))

0.6719511375183997
21.203593877554553
56.981171761353735


In [ ]:
from sklearn.kernel_ridge import KernelRidge
ridge_kernel = make_pipeline(StandardScaler(), KernelRidge(kernel = 'rbf'))
param_grid_ridgeK = {'kernelridge__alpha': np.linspace(0.01, 0.3, 10)}

grid_ridgeK = GridSearchCV(ridge_kernel, param_grid_ridgeK, cv=KFold(5), return_train_score=True, scoring=('neg_root_mean_squared_error'))
grid_ridgeK.fit(X_train_4_value, value_train.values.ravel())
grid_ridgeK.score(X_test[features_selected_value], value_test)
value_test_predicted = grid_ridgeK.predict(X_test[features_selected_value])
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score, r2_score, mean_absolute_error, mean_squared_error
print(r2_score(value_test, value_test_predicted))
print(mean_absolute_error(value_test, value_test_predicted))
print(np.sqrt(mean_squared_error(value_test, value_test_predicted)))


-59.76451270731608

##### (2) Classification

In [ ]:
import numpy as np
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score, cross_validate, GridSearchCV, KFold
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score, r2_score

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split,ShuffleSplit
from scipy.stats import uniform


SVC = make_pipeline(StandardScaler(), SVC())
param_grid_svc = {'svc__C': range(80, 95, 1)}

scoring = {'F1': make_scorer(f1_score),
           'Accuracy': make_scorer(accuracy_score),  
           'Precision':make_scorer(precision_score),
           'Recall': make_scorer(recall_score)}

grid_svc = GridSearchCV(SVC, param_grid_svc, cv=KFold(5), return_train_score=True, scoring=scoring, refit = 'F1')
grid_svc.fit(X_train_4_class, class_train.values.ravel())
grid_svc.score(X_test[features_selected], class_test)

0.8646864686468646

In [ ]:
grid_svc.best_params_

{'svc__C': 81}

In [ ]:
class_test_predicted = grid_svc.predict(X_test[features_selected])

In [ ]:
#aug
print(f1_score(class_test, class_test_predicted, average = 'weighted'))
print(accuracy_score(class_test, class_test_predicted))
print(precision_score(class_test, class_test_predicted))
print(recall_score(class_test, class_test_predicted))
print(f1_score(class_test, class_test_predicted))

0.9294145776726568
0.9291882556131261
0.8562091503267973
0.8733333333333333
0.8646864686468646


In [ ]:
print(accuracy_score(class_test, class_test_predicted))
print(precision_score(class_test, class_test_predicted))
print(recall_score(class_test, class_test_predicted))
print(f1_score(class_test, class_test_predicted))

0.92573402417962
0.8407643312101911
0.88
0.8599348534201954


##### (3) Evaluate on subgroup

In [ ]:
def evaluate_A_site(element):
  # Evaluate on the subgroup of Ba in the A site
  index_Ba = data_composition.index[np.logical_or(np.logical_or(data_composition['A site #1'] == element, data_composition['A site #2'] == element), data_composition['A site #3'] == element)
  ].tolist()

  Ba_subgroup = X_test[X_test.index.isin(index_Ba)]

  class_Ba_predicted = grid_svc.predict(Ba_subgroup[features_selected])
  print(f1_score(class_test[X_test.index.isin(index_Ba)], class_Ba_predicted, average = 'weighted'))
  print(accuracy_score(class_test[X_test.index.isin(index_Ba)], class_Ba_predicted))
  print(precision_score(class_test[X_test.index.isin(index_Ba)], class_Ba_predicted))
  print(recall_score(class_test[X_test.index.isin(index_Ba)], class_Ba_predicted))
  print(f1_score(class_test[X_test.index.isin(index_Ba)], class_Ba_predicted))

In [ ]:
# Evaluate on the subgroup of Ba in the A site
index_Ba = data_composition.index[np.logical_or(np.logical_or(data_composition['A site #1'] == 'Ba', data_composition['A site #2'] == 'Ba'), data_composition['A site #3'] == 'Ba')
].tolist()

Ba_subgroup = X_test[X_test.index.isin(index_Ba)]

class_Ba_predicted = grid_svc.predict(Ba_subgroup[features_selected])
print(f1_score(class_test[X_test.index.isin(index_Ba)], class_Ba_predicted, average = 'weighted'))
print(accuracy_score(class_test[X_test.index.isin(index_Ba)], class_Ba_predicted))
print(precision_score(class_test[X_test.index.isin(index_Ba)], class_Ba_predicted))
print(recall_score(class_test[X_test.index.isin(index_Ba)], class_Ba_predicted))
print(f1_score(class_test[X_test.index.isin(index_Ba)], class_Ba_predicted))

0.9671052631578948
0.9659090909090909
0.8076923076923077
0.9545454545454546
0.875


In [ ]:
def evaluate_B_site(element):
  index_Mn = data_composition.index[np.logical_or(np.logical_or(data_composition['B site #1'] == element, data_composition['B site #2'] == element), data_composition['B site #3'] == element)].tolist()
  Mn_subgroup = X_test[X_test.index.isin(index_Mn)]

  class_Mn_predicted = grid_svc.predict(Mn_subgroup[features_selected])
  print(f1_score(class_test[X_test.index.isin(index_Mn)], class_Mn_predicted, average = 'weighted'))
  print(accuracy_score(class_test[X_test.index.isin(index_Mn)], class_Mn_predicted))
  print(precision_score(class_test[X_test.index.isin(index_Mn)], class_Mn_predicted))
  print(recall_score(class_test[X_test.index.isin(index_Mn)], class_Mn_predicted))
  print(f1_score(class_test[X_test.index.isin(index_Mn)], class_Mn_predicted))
evaluate_B_site('Fe')
print('\n')
evaluate_B_site('Co')

0.9718694654784
0.9717514124293786
0.9285714285714286
0.9512195121951219
0.9397590361445782


0.920699826723923
0.9216867469879518
0.6470588235294118
0.6111111111111112
0.6285714285714287


In [ ]:
# Evaluate on the subgroup of Mn in the B site

index_Mn = data_composition.index[np.logical_or(np.logical_or(data_composition['B site #1'] == 'Mn', data_composition['B site #2'] == 'Mn'), data_composition['B site #3'] == 'Mn')].tolist()
Mn_subgroup = X_test[X_test.index.isin(index_Mn)]

class_Mn_predicted = grid_svc.predict(Mn_subgroup[features_selected])
print(f1_score(class_test[X_test.index.isin(index_Mn)], class_Mn_predicted, average = 'weighted'))
print(accuracy_score(class_test[X_test.index.isin(index_Mn)], class_Mn_predicted))
print(precision_score(class_test[X_test.index.isin(index_Mn)], class_Mn_predicted))
print(recall_score(class_test[X_test.index.isin(index_Mn)], class_Mn_predicted))
print(f1_score(class_test[X_test.index.isin(index_Mn)], class_Mn_predicted))

0.9381985555469268
0.9338235294117647
0.68
0.9444444444444444
0.7906976744186047


In [ ]:
def evaluate_A_site_reg(element):
  # Evaluate on the subgroup of Ba in the A site
  index_Ba = data_composition.index[np.logical_or(np.logical_or(data_composition['A site #1'] == element, data_composition['A site #2'] == element), data_composition['A site #3'] == element)
  ].tolist()

  Ba_subgroup = X_test[X_test.index.isin(index_Ba)]

  class_Ba_predicted = grid_svr.predict(Ba_subgroup[features_selected_value])
  print(r2_score(class_test[X_test.index.isin(index_Ba)], class_Ba_predicted))
  print(mean_absolute_error(class_test[X_test.index.isin(index_Ba)], class_Ba_predicted))
  print(np.sqrt(mean_squared_error(class_test[X_test.index.isin(index_Ba)], class_Ba_predicted)))

evaluate_A_site_reg('Ba')
print('\n')
evaluate_A_site_reg('Sr')

-274572.8498590419
147.87090634846237
173.29603235023217


-244184.7706843248
129.83427544769657
149.58065714072234


In [ ]:
def evaluate_B_site_reg(element):
  index_Mn = data_composition.index[np.logical_or(np.logical_or(data_composition['B site #1'] == element, data_composition['B site #2'] == element), data_composition['B site #3'] == element)].tolist()
  Mn_subgroup = X_test[X_test.index.isin(index_Mn)]

  class_Mn_predicted = grid_svr.predict(Mn_subgroup[features_selected_value])

  print(r2_score(class_test[X_test.index.isin(index_Mn)], class_Mn_predicted))
  print(mean_absolute_error(class_test[X_test.index.isin(index_Mn)], class_Mn_predicted))
  print(np.sqrt(mean_squared_error(class_test[X_test.index.isin(index_Mn)], class_Mn_predicted)))
evaluate_B_site_reg('Fe')
print('\n')
evaluate_B_site_reg('Co')

-105716.01794307254
109.1409669896347
137.1704515308153


-236716.17835806354
131.12758824787159
151.2773506191377


##### (4) Calculating CI

In [ ]:
random_state = [0, 100, 200, 300, 400, 500, 600, 700, 800, 900]
#random_state = [42]
def calculate_CI(random_state, X, y_class, y_value):
  from sklearn.svm import SVC
  MAE, RMSE, R2 = [], [], []
  WEIGHTED_F1, ACC, PRECISION, RECALL, F1 = [], [], [], [], []
  WEIGHTED_F1_Ba, WEIGHTED_F1_Sr, WEIGHTED_F1_Fe, WEIGHTED_F1_Co = [],[],[],[]
  MAE_Ba, MAE_Sr, MAE_Fe, MAE_Co = [], [], [], []
  RMSE_Ba, RMSE_Sr, RMSE_Fe, RMSE_Co = [], [], [], []

  for state in random_state:
    X_train, X_test, class_train, class_test = train_test_split(X[features_selected], y_class, test_size=0.3, random_state=state)
    X_train, X_test, value_train, value_test = train_test_split(X[features_selected_value], y_value, test_size=0.3, random_state=state)
    
    SVReg = make_pipeline(StandardScaler(), SVR())
    param_grid_svr = {'svr__C': range(2200, 2300, 10)}
    grid_svr = GridSearchCV(SVReg, param_grid_svr, cv=KFold(5), return_train_score=True, scoring='r2')
    grid_svr.fit(X_train, value_train.values.ravel())
    grid_svr.score(X_test, value_test)
    value_test_predicted_svr = grid_svr.predict(X_test)
    R2.append(r2_score(value_test, value_test_predicted_svr))
    MAE.append(mean_absolute_error(value_test, value_test_predicted_svr))
    RMSE.append(np.sqrt(mean_squared_error(value_test, value_test_predicted_svr)))



    svc = make_pipeline(StandardScaler(), SVC())
    param_grid_svc = {'svc__C': range(75, 95, 1)}
    grid_svc = GridSearchCV(svc, param_grid_svc, cv=KFold(5), return_train_score=True, scoring='f1')
    grid_svc.fit(X_train, class_train.values.ravel())
    grid_svc.score(X_test, class_test)
    class_test_predicted = grid_svc.predict(X_test)
    WEIGHTED_F1.append(f1_score(class_test, class_test_predicted, average = 'weighted'))
    ACC.append(accuracy_score(class_test, class_test_predicted))
    PRECISION.append(precision_score(class_test, class_test_predicted))
    RECALL.append(recall_score(class_test, class_test_predicted))
    F1.append(f1_score(class_test, class_test_predicted))


    

    def evaluate(element, res, site1,site2,site3, model, metric, y):

      index_Ba = data_composition.index[np.logical_or(np.logical_or(data_composition[site1] == element, data_composition[site2] == element), data_composition[site3] == element)
      ].tolist()
      Ba_subgroup = X_test[X_test.index.isin(index_Ba)]

      class_Ba_predicted = model.predict(Ba_subgroup)
      if metric == 'f1_score':
        res.append(f1_score(y[X_test.index.isin(index_Ba)], class_Ba_predicted, average = 'weighted'))
      elif metric == 'mae':
        res.append(mean_absolute_error(y[X_test.index.isin(index_Ba)], class_Ba_predicted))
      elif metric == 'rmse':
        res.append(np.sqrt(mean_squared_error(y[X_test.index.isin(index_Ba)], class_Ba_predicted)))

    evaluate('Ba', WEIGHTED_F1_Ba, 'A site #1', 'A site #2', 'A site #3', grid_svc, 'f1_score', class_test)
    evaluate('Sr', WEIGHTED_F1_Sr, 'A site #1', 'A site #2', 'A site #3', grid_svc, 'f1_score', class_test)
    evaluate('Fe', WEIGHTED_F1_Fe, 'B site #1', 'B site #2', 'B site #3', grid_svc, 'f1_score', class_test)
    evaluate('Co', WEIGHTED_F1_Co, 'B site #1', 'B site #2', 'B site #3', grid_svc, 'f1_score', class_test)
    evaluate('Ba', MAE_Ba, 'A site #1', 'A site #2', 'A site #3', grid_svr, 'mae', value_test)
    evaluate('Sr', MAE_Sr, 'A site #1', 'A site #2', 'A site #3', grid_svr, 'mae', value_test)
    evaluate('Fe', MAE_Fe, 'B site #1', 'B site #2', 'B site #3', grid_svr, 'mae', value_test)
    evaluate('Co', MAE_Co, 'B site #1', 'B site #2', 'B site #3', grid_svr, 'mae', value_test)
    evaluate('Ba', RMSE_Ba, 'A site #1', 'A site #2', 'A site #3', grid_svr, 'rmse', value_test)
    evaluate('Sr', RMSE_Sr, 'A site #1', 'A site #2', 'A site #3', grid_svr, 'rmse', value_test)
    evaluate('Fe', RMSE_Fe, 'B site #1', 'B site #2', 'B site #3', grid_svr, 'rmse', value_test)
    evaluate('Co', RMSE_Co, 'B site #1', 'B site #2', 'B site #3', grid_svr, 'rmse', value_test)

  return  MAE, RMSE, R2, WEIGHTED_F1, ACC, PRECISION, RECALL, F1, WEIGHTED_F1_Ba, WEIGHTED_F1_Sr, WEIGHTED_F1_Fe, WEIGHTED_F1_Co, MAE_Ba, MAE_Sr, MAE_Fe, MAE_Co, RMSE_Ba, RMSE_Sr, RMSE_Fe, RMSE_Co

In [ ]:
MAE, RMSE, R2, WEIGHTED_F1, ACC, PRECISION, RECALL, F1, WEIGHTED_F1_Ba, WEIGHTED_F1_Sr, WEIGHTED_F1_Fe, WEIGHTED_F1_Co, MAE_Ba, MAE_Sr, MAE_Fe, MAE_Co, RMSE_Ba, RMSE_Sr, RMSE_Fe, RMSE_Co = calculate_CI(random_state, features_full, target_class, target_value)

In [ ]:
import numpy as np
print((np.mean(np.array(MAE))), np.std(np.array(MAE))*2)
print(np.mean(np.array(MAE_Ba)), np.std(np.array(MAE_Ba))*2)
print(np.mean(np.array(MAE_Sr)), np.std(np.array(MAE_Sr))*2)
print(np.mean(np.array(MAE_Fe)), np.std(np.array(MAE_Fe))*2)
print(np.mean(np.array(MAE_Co)), np.std(np.array(MAE_Co))*2)

print(np.mean(np.array(WEIGHTED_F1)), np.std(np.array(WEIGHTED_F1))*2)
print(np.mean(np.array(WEIGHTED_F1_Ba)), np.std(np.array(WEIGHTED_F1_Ba))*2)
print(np.mean(np.array(WEIGHTED_F1_Sr)), np.std(np.array(WEIGHTED_F1_Sr))*2)
print(np.mean(np.array(WEIGHTED_F1_Fe)), np.std(np.array(WEIGHTED_F1_Fe))*2)
print(np.mean(np.array(WEIGHTED_F1_Co)), np.std(np.array(WEIGHTED_F1_Co))*2)

print(np.mean(np.array(RMSE)), np.std(np.array(RMSE))*2)
print(np.mean(np.array(RMSE_Ba)), np.std(np.array(RMSE_Ba))*2)
print(np.mean(np.array(RMSE_Sr)), np.std(np.array(RMSE_Sr))*2)
print(np.mean(np.array(RMSE_Fe)), np.std(np.array(RMSE_Fe))*2)
print(np.mean(np.array(RMSE_Co)), np.std(np.array(RMSE_Co))*2)

20.643261209466043 2.280784441769721
28.431975925863572 6.076939745269715
14.939859133127962 3.86985707755885
24.196891263037866 4.827564552137484
18.216044248796255 3.6684009234334356
0.903822790040309 0.0439364370000172
0.9192704623108199 0.05646713476687788
0.9352188833268074 0.053739901657352235
0.9160436246318184 0.0634893198574281
0.9317729459048272 0.0405768868289813
50.58846481976145 15.103105184730246
75.6846804284398 32.09248015194238
26.057934142014613 8.953134616870642
73.45460973153828 30.24472451639533
33.17561668615567 9.444554909788074


In [ ]:
import numpy as np
print((np.mean(np.array(MAE))), np.std(np.array(MAE))*2)
print(np.mean(np.array(MAE_Ba)), np.std(np.array(MAE_Ba))*2)
print(np.mean(np.array(MAE_Sr)), np.std(np.array(MAE_Sr))*2)
print(np.mean(np.array(MAE_Fe)), np.std(np.array(MAE_Fe))*2)
print(np.mean(np.array(MAE_Co)), np.std(np.array(MAE_Co))*2)

print(np.mean(np.array(WEIGHTED_F1)), np.std(np.array(WEIGHTED_F1))*2)
print(np.mean(np.array(WEIGHTED_F1_Ba)), np.std(np.array(WEIGHTED_F1_Ba))*2)
print(np.mean(np.array(WEIGHTED_F1_Sr)), np.std(np.array(WEIGHTED_F1_Sr))*2)
print(np.mean(np.array(WEIGHTED_F1_Fe)), np.std(np.array(WEIGHTED_F1_Fe))*2)
print(np.mean(np.array(WEIGHTED_F1_Co)), np.std(np.array(WEIGHTED_F1_Co))*2)

print(np.mean(np.array(RMSE)), np.std(np.array(RMSE))*2)
print(np.mean(np.array(RMSE_Ba)), np.std(np.array(RMSE_Ba))*2)
print(np.mean(np.array(RMSE_Sr)), np.std(np.array(RMSE_Sr))*2)
print(np.mean(np.array(RMSE_Fe)), np.std(np.array(RMSE_Fe))*2)
print(np.mean(np.array(RMSE_Co)), np.std(np.array(RMSE_Co))*2)



20.643261209466043 2.280784441769721
28.431975925863572 6.076939745269715
14.939859133127962 3.86985707755885
24.196891263037866 4.827564552137484
18.216044248796255 3.6684009234334356
0.903822790040309 0.0439364370000172
0.9192704623108199 0.05646713476687788
0.9352188833268074 0.053739901657352235
0.9160436246318184 0.0634893198574281
0.9317729459048272 0.0405768868289813
50.58846481976145 15.103105184730246
75.6846804284398 32.09248015194238
26.057934142014613 8.953134616870642
73.45460973153828 30.24472451639533
33.17561668615567 9.444554909788074


#### 2.2 GBDT

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split,ShuffleSplit
from sklearn.preprocessing import StandardScaler
from scipy.stats import uniform
from imblearn.pipeline import Pipeline
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score, precision_score, r2_score, recall_score,mean_absolute_error
from sklearn.model_selection import cross_val_score, cross_validate, GridSearchCV, KFold
from sklearn.decomposition import PCA
from sklearn.feature_selection import f_regression, f_classif, RFE, VarianceThreshold, chi2, SelectKBest, SelectFromModel
from sklearn.ensemble import GradientBoostingClassifier

##### (1) Regression

In [ ]:
gbdt_reg = make_pipeline(StandardScaler(), GradientBoostingRegressor())
param_grid_reg = {'gradientboostingregressor__alpha':  np.linspace(0.001, 0.01, 5)}
grid_reg = GridSearchCV(gbdt_reg, param_grid_reg, cv=KFold(5), return_train_score=True,scoring= 'r2')
grid_reg.fit(X_train_4_value, value_train.values.ravel())
grid_reg.score(X_test[features_selected_value], value_test)


0.6521863323589769

In [ ]:
grid_reg.best_params_

{'gradientboostingregressor__alpha': 0.001,
 'gradientboostingregressor__n_estimators': 350}

In [ ]:
value_test_predicted_reg= grid_reg.predict(X_test[features_selected_value])
print(r2_score(value_test, value_test_predicted_reg))
print(mean_absolute_error(value_test, value_test_predicted_reg))
print(np.sqrt(mean_squared_error(value_test, value_test_predicted_reg)))

0.6521863323589769
28.095110093551593
58.6726129559543


##### (2) Classification

In [ ]:
gbdt_clf = make_pipeline(StandardScaler(), GradientBoostingClassifier())
param_grid_clf = {'gradientboostingclassifier__ccp_alpha':np.linspace(0,0.001, 5)}
grid_clf = GridSearchCV(gbdt_clf, param_grid_clf, cv=KFold(5), return_train_score=True,scoring='f1')
grid_clf.fit(X_train_4_class, class_train.values.ravel())
grid_clf.score(X_test[features_selected], class_test)

0.8294314381270903

In [ ]:
class_test_predicted = grid_clf.predict(X_test[features_selected])
print(accuracy_score(class_test, class_test_predicted))
print(precision_score(class_test, class_test_predicted))
print(recall_score(class_test, class_test_predicted))
print(f1_score(class_test, class_test_predicted))

0.9119170984455959
0.8322147651006712
0.8266666666666667
0.8294314381270903


##### (2) Evaluate on subgroup

In [ ]:
random_state = [0, 100, 200, 300, 400, 500, 600, 700, 800, 900]
def calculate_CI(random_state, X, y_class, y_value):
  MAE, RMSE, R2 = [], [], []
  WEIGHTED_F1, ACC, PRECISION, RECALL, F1 = [], [], [], [], []
  WEIGHTED_F1_Ba, WEIGHTED_F1_Sr, WEIGHTED_F1_Fe, WEIGHTED_F1_Co = [],[],[],[]
  MAE_Ba, MAE_Sr, MAE_Fe, MAE_Co = [], [], [], []
  RMSE_Ba, RMSE_Sr, RMSE_Fe, RMSE_Co = [], [], [], []

  for state in random_state:
    X_train, X_test, class_train, class_test = train_test_split(X[features_selected], y_class, test_size=0.3, random_state=state)
    X_train, X_test, value_train, value_test = train_test_split(X[features_selected_value], y_value, test_size=0.3, random_state=state)
    
    gbdt_reg = make_pipeline(StandardScaler(), GradientBoostingRegressor())
    param_grid_reg = {'gradientboostingregressor__alpha':  np.linspace(0.001, 0.01, 5)}
    grid_reg = GridSearchCV(gbdt_reg, param_grid_reg, cv=KFold(5), return_train_score=True, scoring='r2')
    grid_reg.fit(X_train, value_train.values.ravel())
    grid_reg.score(X_test, value_test)
    value_test_predicted_reg = grid_reg.predict(X_test)
    R2.append(r2_score(value_test, value_test_predicted_reg))
    MAE.append(mean_absolute_error(value_test, value_test_predicted_reg))
    RMSE.append(np.sqrt(mean_squared_error(value_test, value_test_predicted_reg)))
    
    gbdt_clf = make_pipeline(StandardScaler(), GradientBoostingClassifier())
    param_grid_clf = {'gradientboostingclassifier__ccp_alpha':np.linspace(0,0.001, 5)}
    grid_clf = GridSearchCV(gbdt_clf, param_grid_clf, cv=KFold(5), return_train_score=True,scoring='f1')
    grid_clf.fit(X_train, class_train.values.ravel())
    grid_clf.score(X_test, class_test)
    class_test_predicted = grid_clf.predict(X_test)
    WEIGHTED_F1.append(f1_score(class_test, class_test_predicted, average = 'weighted'))
    ACC.append(accuracy_score(class_test, class_test_predicted))
    PRECISION.append(precision_score(class_test, class_test_predicted))
    RECALL.append(recall_score(class_test, class_test_predicted))
    F1.append(f1_score(class_test, class_test_predicted))


    

    def evaluate(element, res, site1,site2,site3, model, metric, y):

      index_Ba = data_composition.index[np.logical_or(np.logical_or(data_composition[site1] == element, data_composition[site2] == element), data_composition[site3] == element)
      ].tolist()
      Ba_subgroup = X_test[X_test.index.isin(index_Ba)]

      class_Ba_predicted = model.predict(Ba_subgroup)
      if metric == 'f1_score':
        res.append(f1_score(y[X_test.index.isin(index_Ba)], class_Ba_predicted, average = 'weighted'))
      elif metric == 'mae':
        res.append(mean_absolute_error(y[X_test.index.isin(index_Ba)], class_Ba_predicted))
      elif metric == 'rmse':
        res.append(np.sqrt(mean_squared_error(y[X_test.index.isin(index_Ba)], class_Ba_predicted)))

    evaluate('Ba', WEIGHTED_F1_Ba, 'A site #1', 'A site #2', 'A site #3', grid_clf, 'f1_score', class_test)
    evaluate('Sr', WEIGHTED_F1_Sr, 'A site #1', 'A site #2', 'A site #3', grid_clf, 'f1_score', class_test)
    evaluate('Fe', WEIGHTED_F1_Fe, 'B site #1', 'B site #2', 'B site #3', grid_clf, 'f1_score', class_test)
    evaluate('Co', WEIGHTED_F1_Co, 'B site #1', 'B site #2', 'B site #3', grid_clf, 'f1_score', class_test)
    evaluate('Ba', MAE_Ba, 'A site #1', 'A site #2', 'A site #3', grid_reg, 'mae', value_test)
    evaluate('Sr', MAE_Sr, 'A site #1', 'A site #2', 'A site #3', grid_reg, 'mae', value_test)
    evaluate('Fe', MAE_Fe, 'B site #1', 'B site #2', 'B site #3', grid_reg, 'mae', value_test)
    evaluate('Co', MAE_Co, 'B site #1', 'B site #2', 'B site #3', grid_reg, 'mae', value_test)
    evaluate('Ba', RMSE_Ba, 'A site #1', 'A site #2', 'A site #3', grid_reg, 'rmse', value_test)
    evaluate('Sr', RMSE_Sr, 'A site #1', 'A site #2', 'A site #3', grid_reg, 'rmse', value_test)
    evaluate('Fe', RMSE_Fe, 'B site #1', 'B site #2', 'B site #3', grid_reg, 'rmse', value_test)
    evaluate('Co', RMSE_Co, 'B site #1', 'B site #2', 'B site #3', grid_reg, 'rmse', value_test)

  return  MAE, RMSE, R2, WEIGHTED_F1, ACC, PRECISION, RECALL, F1, WEIGHTED_F1_Ba, WEIGHTED_F1_Sr, WEIGHTED_F1_Fe, WEIGHTED_F1_Co, MAE_Ba, MAE_Sr, MAE_Fe, MAE_Co, RMSE_Ba, RMSE_Sr, RMSE_Fe, RMSE_Co

In [ ]:
MAE, RMSE, R2, WEIGHTED_F1, ACC, PRECISION, RECALL, F1, WEIGHTED_F1_Ba, WEIGHTED_F1_Sr, WEIGHTED_F1_Fe, WEIGHTED_F1_Co, MAE_Ba, MAE_Sr, MAE_Fe, MAE_Co, RMSE_Ba, RMSE_Sr, RMSE_Fe, RMSE_Co = calculate_CI(random_state, features_full, target_class, target_value)

In [ ]:
import numpy as np
print((np.mean(np.array(MAE))), np.std(np.array(MAE))*2)
print(np.mean(np.array(MAE_Ba)), np.std(np.array(MAE_Ba))*2)
print(np.mean(np.array(MAE_Sr)), np.std(np.array(MAE_Sr))*2)
print(np.mean(np.array(MAE_Fe)), np.std(np.array(MAE_Fe))*2)
print(np.mean(np.array(MAE_Co)), np.std(np.array(MAE_Co))*2)

print(np.mean(np.array(WEIGHTED_F1)), np.std(np.array(WEIGHTED_F1))*2)
print(np.mean(np.array(WEIGHTED_F1_Ba)), np.std(np.array(WEIGHTED_F1_Ba))*2)
print(np.mean(np.array(WEIGHTED_F1_Sr)), np.std(np.array(WEIGHTED_F1_Sr))*2)
print(np.mean(np.array(WEIGHTED_F1_Fe)), np.std(np.array(WEIGHTED_F1_Fe))*2)
print(np.mean(np.array(WEIGHTED_F1_Co)), np.std(np.array(WEIGHTED_F1_Co))*2)

print(np.mean(np.array(RMSE)), np.std(np.array(RMSE))*2)
print(np.mean(np.array(RMSE_Ba)), np.std(np.array(RMSE_Ba))*2)
print(np.mean(np.array(RMSE_Sr)), np.std(np.array(RMSE_Sr))*2)
print(np.mean(np.array(RMSE_Fe)), np.std(np.array(RMSE_Fe))*2)
print(np.mean(np.array(RMSE_Co)), np.std(np.array(RMSE_Co))*2)

28.764600790164685 3.1271400200779613
37.156165672124914 6.098612457200731
25.071615839975117 4.543735569398987
33.8058277862451 6.665545002677218
22.229133335149413 3.3181332852788405
0.8844558512231284 0.023703176983948804
0.926209424556971 0.03786752162664054
0.9286665683515005 0.028484835598089696
0.8972000591654601 0.03311407596115093
0.9337570052722988 0.03811907396405043
56.291711589748104 12.411175333777521
82.13622472132361 27.579109362879688
36.46134238666031 7.454063887044454
78.31870560200078 26.56279056204345
33.388479556646686 6.1017479150540455


### 3. Rebuild model on subgroup

(1) Group by A site

In [ ]:
index_Ba_in_A_site = np.logical_or(np.logical_or(data_composition['A site #1'] == 'Ba', data_composition['A site #2'] == 'Ba'), data_composition['A site #3'] == 'Ba')
Material_A_site_Ba = data_composition.loc[index_Ba_in_A_site]
data_Ba = Material_A_site_Ba.merge(data, 'inner', on='Material Composition')

features_full_Ba = data_Ba.drop(['index', 'Material Composition', 'EnergyAboveHull', 'Formation_energy'],axis=1)
target_value_Ba = data_Ba[['EnergyAboveHull']]
target_class_Ba = data_Ba[['EnergyAboveHull']]
target_class_Ba[target_class_Ba <= 40] = 1
target_class_Ba[target_class_Ba > 40] = 0


X_train_Ba, X_test_Ba, class_train_Ba, class_test_Ba = train_test_split(features_full_Ba, target_class_Ba, test_size=0.3, random_state=42)
 

In [ ]:
print('Number of \'Stable\' in the training dataset:', len(class_train_Ba[class_train_Ba['EnergyAboveHull'] == 1.0]))
print('Number of \'Unstable\' in the training dataset:', len(class_train_Ba[class_train_Ba['EnergyAboveHull'] == 0.0]))

Number of 'Stable' in the training dataset: 59
Number of 'Unstable' in the training dataset: 358


In [ ]:
X_train_Ba_class = X_train_Ba[features_selected]
X_resampled_Ba, y_resampled_Ba = ros.fit_resample(X_train_Ba_class, class_train_Ba.values.ravel())
print('Number of \'Stable\' after oversampling:', np.count_nonzero(y_resampled_Ba))
print('Number of \'Unstable\' after oversampling:', np.count_nonzero(y_resampled_Ba))

Number of 'Stable' after oversampling: 358
Number of 'Unstable' after oversampling: 358


Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.


In [ ]:

param_grid_svc = {'svc__C': range(35, 45, 1)}

scoring = {'F1': make_scorer(f1_score),
           'Accuracy': make_scorer(accuracy_score),  
           'Precision':make_scorer(precision_score),
           'Recall': make_scorer(recall_score)}

grid_svc = GridSearchCV(SVC, param_grid_svc, cv=KFold(5), return_train_score=True, scoring=scoring, refit = 'F1')
grid_svc.fit(X_resampled_Ba, y_resampled_Ba)
grid_svc.score(X_test_Ba[features_selected], class_test_Ba)

0.7027027027027027

In [ ]:
class_test_predicted_Ba = grid_svc.predict(X_test_Ba[features_selected])
print(accuracy_score(class_test_Ba, class_test_predicted_Ba))
print(precision_score(class_test_Ba, class_test_predicted_Ba))
print(recall_score(class_test_Ba, class_test_predicted_Ba))
print(f1_score(class_test_Ba, class_test_predicted_Ba))

0.9388888888888889
0.7647058823529411
0.65
0.7027027027027027


In [ ]:
grid_search_clf.fit(X_resampled_Ba, y_resampled_Ba)
grid_search_clf.score(X_test_Ba[features_selected], class_test_Ba)

0.75

In [ ]:
class_test_predicted_Ba = grid_search_clf.predict(X_test_Ba[features_selected])
print(accuracy_score(class_test_Ba, class_test_predicted_Ba))
print(precision_score(class_test_Ba, class_test_predicted_Ba))
print(recall_score(class_test_Ba, class_test_predicted_Ba))
print(f1_score(class_test_Ba, class_test_predicted_Ba))

0.9444444444444444
0.75
0.75
0.75


(2) Group by B site

In [ ]:
index_Mn_in_B_site = np.logical_or(np.logical_or(data_composition['B site #1'] == 'Mn', data_composition['B site #2'] == 'Mn'), data_composition['B site #3'] == 'Mn')
Material_B_site_Mn = data_composition.loc[index_Mn_in_B_site]
data_Mn = Material_B_site_Mn.merge(data, 'inner', on='Material Composition')

features_full_Mn = data_Mn.drop(['index', 'Material Composition', 'EnergyAboveHull', 'Formation_energy'],axis=1)
target_value_Mn = data_Mn[['EnergyAboveHull']]
target_class_Mn = data_Mn[['EnergyAboveHull']]
target_class_Mn[target_class_Mn <= 40] = 1
target_class_Mn[target_class_Mn > 40] = 0


X_train_Mn, X_test_Mn, class_train_Mn, class_test_Mn = train_test_split(features_full_Mn, target_class_Mn, test_size=0.3, random_state=42)
 

In [ ]:
print('Number of \'Stable\' in the training dataset:', len(class_train_Mn[class_train_Mn['EnergyAboveHull'] == 1.0]))
print('Number of \'Unstable\' in the training dataset:', len(class_train_Mn[class_train_Mn['EnergyAboveHull'] == 0.0]))

X_train_Mn_class = X_train_Mn[features_selected]
X_resampled_Mn, y_resampled_Mn = ros.fit_resample(X_train_Mn_class, class_train_Mn.values.ravel())
print('Number of \'Stable\' after oversampling:', np.count_nonzero(y_resampled_Mn))
print('Number of \'Unstable\' after oversampling:', np.count_nonzero(y_resampled_Mn))

Number of 'Stable' in the training dataset: 55
Number of 'Unstable' in the training dataset: 262
Number of 'Stable' after oversampling: 262
Number of 'Unstable' after oversampling: 262


Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.


In [ ]:
param_grid_svc = {'svc__C': range(35, 45, 1)}

scoring = {'F1': make_scorer(f1_score),
           'Accuracy': make_scorer(accuracy_score),  
           'Precision':make_scorer(precision_score),
           'Recall': make_scorer(recall_score)}

grid_svc = GridSearchCV(SVC, param_grid_svc, cv=KFold(5), return_train_score=True, scoring=scoring, refit = 'F1')
grid_svc.fit(X_resampled_Mn, y_resampled_Mn)
grid_svc.score(X_test_Mn[features_selected], class_test_Mn)

0.7540983606557378

In [ ]:
class_test_predicted_Mn = grid_svc.predict(X_test_Mn[features_selected])
print(accuracy_score(class_test_Mn, class_test_predicted_Mn))
print(precision_score(class_test_Mn, class_test_predicted_Mn))
print(recall_score(class_test_Mn, class_test_predicted_Mn))
print(f1_score(class_test_Mn, class_test_predicted_Mn))

0.8897058823529411
0.71875
0.7931034482758621
0.7540983606557378


In [ ]:
grid_search_clf.fit(X_resampled_Mn, y_resampled_Mn)
grid_search_clf.score(X_test_Mn[features_selected], class_test_Mn)

0.71875

In [ ]:
class_test_predicted_Mn = grid_search_clf.predict(X_test_Mn[features_selected])
print(accuracy_score(class_test_Mn, class_test_predicted_Mn))
print(precision_score(class_test_Mn, class_test_predicted_Mn))
print(recall_score(class_test_Mn, class_test_predicted_Mn))
print(f1_score(class_test_Mn, class_test_predicted_Mn))

0.8676470588235294
0.6571428571428571
0.7931034482758621
0.71875
